In [1]:
!pip install transformers 

     |████████████████████████████████| 4.0 MB 5.3 MB/s 
     |████████████████████████████████| 596 kB 9.1 MB/s 
     |████████████████████████████████| 77 kB 2.4 MB/s 
     |████████████████████████████████| 895 kB 13.5 MB/s 
     |████████████████████████████████| 6.6 MB 28.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
#Import following packages 
import pickle
import pandas as pd
from gensim.models import Word2Vec
from scipy.spatial.distance import cosine
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import tensorflow as tf
import torch
from transformers import BertTokenizer, BertModel
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler


In [3]:
#import lemma 
from google.colab import drive
drive.mount('/content/drive')
pickle.dump

Mounted at /content/drive


<function _pickle.dump>

In [4]:
pick_read = open('drive/My Drive/Lemma.pkl','rb')
data = pickle.load(pick_read)
pick_read.close()

In [5]:
data1 = pd.DataFrame(data, columns=['text'])

In [6]:
data1.head()

,text
0,car wonder enlighten car see day door sport ca...
1,clock poll final final clock report accelerati...
2,question folk mac plus finally give ghost week...
3,weitek nntp post host amber ssd csd harris com...
4,shuttle launch question article cowcb world st...


In [7]:
#Generate corpus_gen 
corpus_gen = [text.split() for text in data1['text']]

**Part 1: Word2vec**

In [8]:
model = Word2Vec(corpus_gen, size=100, min_count=566)
#What is it doing? The model learns meaningful relations and encodes the relatedness into vector similarity, in this case words with total frequency lower than 566


In [9]:
model.save('drive/My Drive/word2vec.model')

In [10]:
#Print sorted features 
model.wv.vocab.keys()

dict_keys(['car', 'wonder', 'see', 'day', 'look', 'early', 'call', 'small', 'body', 'know', 'model', 'year', 'history', 'info', 'mail', 'thank', 'bring', 'report', 'nntp', 'post', 'host', 'washington', 'edu', 'number', 'experience', 'send', 'message', 'speed', 'rate', 'add', 'card', 'disk', 'request', 'network', 'base', 'answer', 'guy', 'question', 'mac', 'give', 'start', 'life', 'way', 'new', 'machine', 'bit', 'maybe', 'anybody', 'expect', 'hear', 'suppose', 'access', 'price', 'line', 'like', 'go', 'display', 'probably', 'get', 'feel', 'well', 'great', 'good', 'opinion', 'people', 'use', 'take', 'size', 'money', 'hit', 'real', 'play', 'figure', 'actually', 'advance', 'email', 'news', 'time', 'truth', 'com', 'uucp', 'write', 'article', 'chip', 'far', 'low', 'level', 'stuff', 'pretty', 'nice', 'require', 'address', 'phone', 'information', 'thing', 'person', 'sense', 'world', 'clear', 'memory', 'error', 'yes', 'quote', 'software', 'check', 'right', 'value', 'set', 'code', 'tell', 'second

In [11]:
 model.wv['car']
 

array([-8.8793063e-01,  1.6529957e+00,  8.5118890e-01, -2.2647837e-01,
        1.2928505e+00, -1.3342566e+00,  1.8605837e-01,  4.8897001e-01,
        6.3142842e-01, -4.9925509e-01, -7.6401913e-01,  1.1021389e+00,
       -9.7026670e-01, -2.7842116e-01, -1.3067150e+00, -1.9602215e-01,
        1.5988441e+00, -3.2006297e-01, -1.0350718e+00,  1.7484404e+00,
        5.6522060e-02, -1.0241311e+00,  1.9352800e-01,  1.6362010e+00,
        8.2681865e-01, -7.2784615e-01,  1.0019224e+00, -8.6016631e-01,
        1.2712419e+00, -7.0579126e-02, -5.6197923e-01,  3.3750582e-02,
        9.5684238e-02,  1.1731559e+00,  3.5399768e-01,  4.3940556e-01,
       -5.9952939e-01,  6.3999277e-01, -2.9493296e-01,  3.5995957e-01,
       -2.5494751e-01,  9.5584150e-03,  1.6415122e+00, -9.3547279e-01,
       -1.6232264e+00,  4.9131513e-01, -1.4438710e-01, -1.0328842e-03,
        1.1003612e+00,  4.7199856e-02,  1.1268346e+00,  8.4888041e-01,
        1.0897775e+00,  9.2178024e-02,  5.5370986e-01,  1.3303546e+00,
      

In [12]:
model.wv.most_similar('car')

[('bike', 0.5985946655273438),
 ('buy', 0.535148024559021),
 ('light', 0.5080544948577881),
 ('figure', 0.48679161071777344),
 ('speed', 0.48030927777290344),
 ('nice', 0.45413967967033386),
 ('hit', 0.45118317008018494),
 ('get', 0.45045384764671326),
 ('friend', 0.4493974447250366),
 ('turn', 0.4472292363643646)]

In [13]:
model.wv.most_similar(positive=['bike','machine'], topn=1)

[('fast', 0.6715289950370789)]

In [15]:
model.wv.most_similar(positive=['hardware', 'wrong'], topn=1)
model.wv.most_similar(positive=['space', 'nasa'], topn=1)
model.wv.most_similar(positive=['city', 'child'], topn=5)

[('woman', 0.7533478140830994),
 ('armenians', 0.6520825624465942),
 ('bring', 0.6362184882164001),
 ('kill', 0.6202683448791504),
 ('country', 0.562999963760376)]

In [17]:
import numpy as np
corpus_gen2 = []
for doc in corpus_gen:
  embb = []
  for words in doc:
     if list(model.wv.vocab.keys()).count(words) > 0: embb.append(model.wv[words])
  if len(embb) != 0: corpus_gen2.append(np.mean(embb, axis = 0)) #Averaging the word embedding 



In [19]:
from pandas.core.frame import DataFrame
WordtoVecModel = DataFrame(corpus_gen2)
WordtoVecModel

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.215053,0.377721,0.371721,-0.028690,0.256952,-0.351786,0.025471,0.101180,0.254704,0.136424,...,-0.313591,-0.040760,0.291012,-0.189297,-0.592756,0.594529,-0.535827,-0.258423,-0.260391,-0.236386
1,-0.280870,0.225981,0.180845,0.190747,-0.109120,0.175319,0.049394,-0.808815,-0.112465,-0.202750,...,-0.614012,0.010603,0.003514,0.240820,0.153100,0.474225,-0.323871,0.250113,-0.234336,0.069415
2,-0.190654,-0.006652,0.101558,-0.006926,0.110742,-0.005728,0.116449,0.044235,-0.077686,-0.033120,...,-0.519285,-0.216033,0.068935,-0.197484,-0.037838,0.203313,-0.252762,0.289611,-0.159614,0.273443
3,-0.118284,0.288318,0.098523,0.124358,0.118506,0.077565,0.099886,-0.668313,-0.027845,-0.303632,...,-0.110571,0.173490,0.029877,0.190114,0.231378,0.048586,-0.132265,0.118927,-0.319643,0.023362
4,-0.038382,0.013891,-0.014789,0.236653,0.205435,0.287127,0.215267,-0.239760,0.141431,-0.187108,...,-0.091991,0.003026,-0.044138,-0.116391,0.130448,0.062141,0.047850,0.224775,-0.118669,0.235762
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11294,0.077836,-0.203733,0.272355,0.238867,0.085503,-0.037766,-0.066245,0.226190,0.042022,0.063410,...,-0.069517,-0.303971,-0.107331,-0.332080,-0.156884,0.131370,-0.101308,-0.149044,-0.048312,-0.067512
11295,-0.214699,0.008849,-0.239383,0.083342,0.174039,0.159579,-0.068894,-0.272660,-0.097077,0.132566,...,-0.790268,0.215854,0.059691,0.263771,-0.094487,0.240895,-0.047801,0.166520,-0.246084,0.355070
11296,-0.079540,-0.156395,0.085540,0.002461,0.215282,-0.047930,0.011583,-0.015590,-0.267388,-0.280716,...,-0.144800,-0.023815,-0.026040,-0.151732,0.245138,-0.129288,0.072774,0.006732,-0.129945,-0.110203
11297,0.069076,0.063148,0.209283,0.117738,-0.105806,0.371877,0.268955,-0.369431,-0.066259,-0.048016,...,0.037051,-0.102307,0.137835,0.011291,0.183294,0.333966,-0.122686,0.247531,0.055132,-0.198999


In [20]:
#Store result 
WordtoVecModel.to_pickle('/content/drive/MyDrive/WordtoVecModel.pkl')

In [23]:
from sklearn.metrics.pairwise import cosine_similarity
print(cosine_similarity(corpus_gen2))

[[ 0.99999994  0.12451196  0.36265814 ...  0.02710095 -0.06532171
   0.11612538]
 [ 0.12451196  0.9999998   0.36431178 ...  0.17215036  0.37916878
   0.6503855 ]
 [ 0.36265814  0.36431178  1.0000001  ...  0.29835856 -0.01573658
   0.23676561]
 ...
 [ 0.02710095  0.17215036  0.29835856 ...  0.9999998   0.16796865
   0.14068824]
 [-0.06532171  0.37916878 -0.01573658 ...  0.16796865  1.0000002
   0.29660147]
 [ 0.11612538  0.6503855   0.23676561 ...  0.14068824  0.29660147
   0.9999999 ]]


**Part 2: Doc2vec**

In [26]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(corpus_gen)]

#Run gensim doc2vec model on corpus_gen
modelTag = Doc2Vec(documents, vector_size=100, min_count=566)


In [31]:
#Determine the embedding representation for the first document
embRep = modelTag.infer_vector(documents[0][0])
embRep

array([-1.87500976e-02,  3.18600647e-02,  7.29009695e-03,  6.46344647e-02,
        6.26692269e-03, -4.12317971e-03,  3.94292586e-02, -5.51914657e-03,
        1.18801355e-01,  4.43402119e-02,  2.51188911e-02,  3.62535119e-02,
        1.11461235e-02,  7.38928467e-02, -8.57156664e-02,  4.39159153e-03,
        9.20809060e-03, -7.39838481e-02, -7.27304220e-02,  5.25283441e-03,
        8.07197299e-03, -3.73229533e-02, -5.35706466e-04, -5.48897125e-03,
        5.08464947e-02, -7.79369026e-02,  4.43070047e-02, -2.16128230e-02,
       -1.30741317e-02,  7.60858320e-03,  7.65404478e-02,  2.77856272e-02,
       -9.10346676e-03,  4.07429449e-02,  7.86557719e-02, -5.41465916e-02,
        1.75033640e-02,  1.05413515e-02, -4.76339795e-02,  6.89431373e-03,
       -2.55985875e-02,  6.65766150e-02, -4.78696153e-02, -5.47786281e-02,
       -1.67071633e-03, -4.85988669e-02, -4.07484770e-02,  2.11355891e-02,
        3.81598733e-02,  5.03415465e-02,  6.88680336e-02,  1.08423899e-03,
        1.92344263e-02, -

In [32]:
modelTag.docvecs.most_similar([embRep],topn=10)

[(0, 0.900983452796936),
 (958, 0.8417046070098877),
 (6997, 0.8274346590042114),
 (3550, 0.8195458650588989),
 (2353, 0.8008109927177429),
 (7920, 0.7942432165145874),
 (10024, 0.7793654203414917),
 (11077, 0.7764190435409546),
 (8266, 0.7748289108276367),
 (5371, 0.7729223370552063)]

In [34]:
final_corpus = [modelTag.infer_vector(doc) for doc, id in documents]
DoctoVecModel = DataFrame(final_corpus)
DoctoVecModel


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.005645,0.046815,-0.018793,0.055360,0.023797,0.010450,0.038887,-0.042957,0.084889,0.020042,...,-0.074560,0.047803,0.080647,-0.027933,-0.103370,0.053080,-0.142757,0.002828,0.054576,-0.009183
1,-0.013568,-0.021447,0.015584,0.021895,0.000755,0.020051,-0.029422,-0.064271,-0.039760,-0.018019,...,-0.072035,-0.015851,0.016498,-0.012147,0.021840,0.047873,-0.009756,0.041115,0.005236,-0.013632
2,-0.117957,0.041219,0.004135,0.006696,0.045335,0.042304,0.069426,0.044757,-0.053162,-0.070033,...,-0.083095,-0.038431,0.031402,-0.089627,0.066971,0.159629,-0.045100,0.189874,-0.043229,0.187280
3,-0.046177,0.026505,-0.049191,-0.017352,-0.015124,0.026528,0.001583,-0.012283,0.019390,-0.002893,...,0.018480,0.001379,0.072117,-0.030940,0.019752,-0.020079,-0.034002,-0.021716,-0.032425,-0.089250
4,-0.032975,-0.074159,-0.094132,0.022086,0.056220,0.028930,0.009201,0.021263,0.054387,0.029892,...,-0.035712,-0.026165,0.028687,-0.110765,-0.011389,0.007606,0.009405,0.105671,-0.023769,-0.024196
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11309,-0.002623,-0.095315,-0.000374,0.076682,-0.014542,0.078573,0.011545,0.023396,0.069595,-0.017052,...,-0.096606,-0.004002,0.026277,-0.078514,-0.070995,0.050245,-0.113033,-0.017539,0.024110,0.060815
11310,-0.063329,0.044970,-0.113010,0.098026,0.007942,0.115232,-0.010859,0.032104,0.055078,-0.066952,...,-0.088540,0.008157,0.027311,-0.037136,-0.071176,0.093960,-0.110533,0.041914,0.046862,0.037878
11311,-0.026193,-0.044067,-0.053784,-0.017354,0.026252,-0.007880,-0.043287,0.017711,-0.028586,-0.044207,...,-0.036863,0.014714,0.037265,-0.061424,0.000938,-0.032455,-0.035243,0.028601,0.019755,-0.069880
11312,-0.034118,-0.038308,-0.040517,0.068989,0.012493,0.083681,0.089102,0.030632,0.037369,0.032730,...,-0.074745,0.004763,0.086950,-0.104554,-0.013791,0.042907,-0.073515,0.070425,0.023987,-0.093237


In [35]:
#Store dataframe
DoctoVecModel.to_pickle('/content/drive/MyDrive/DoctoVecModel.pkl')